<a href="https://colab.research.google.com/github/AzizKazi/weatherwise-Aziz-kazi/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pyinputplus matplotlib requests


Import required packages

In [17]:
import os
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip

## 🌤️ Weather Data Functions to fetch weather details


In [20]:
def fetch_weather_for_city(city_name, num_forecast_days=3):
    """
    Fetches weather data for a specific city and number of forecast days.
    - Constructs the API URL using the city_name, e.g., "Sydney".
    - num_forecast_days determines how many days of forecast data to retrieve.
    - Sends a GET request to wttr.in and parses the JSON response.
    """
    api_endpoint = f"https://wttr.in/Sydney?format=j1"
    try:
        http_response = requests.get(api_endpoint)
        http_response.raise_for_status()
        weather_json = http_response.json()
        weather_json['weather'] = weather_json['weather'][:num_forecast_days]
        return weather_json
    except Exception as exception:
        print(f"Error occurred while fetching weather data: {exception}")
        return None

Analyzing the key data in json format

In [19]:
weather_data = get_weather_data(forecast_days=3)
if weather_data:
    print(weather_data.keys())
else:
    print("No weather data received.")

dict_keys(['current_condition', 'nearest_area', 'request', 'weather'])


## 📊 Visualisation Functions

function to display maximum and minimum temperature

In [10]:
def plot_temperature_trends(sydney_weather, display_mode='display'):
    """
    Visualize Sydney's temperature trends based on the provided weather data.

    - sydney_weather: dictionary containing 'weather' list from fetch_weather_for_city() function.
    - display_mode: either 'display' to show the plot immediately or 'figure' to return the figure object.
    """
    date_list = []
    max_temperature_values = []
    min_temperature_values = []

    for daily_record in sydney_weather['weather']:
        date_list.append(daily_record['date'])
        max_temperature_values.append(float(daily_record['maxtempC']))
        min_temperature_values.append(float(daily_record['mintempC']))

    fig, axis = plt.subplots()
    axis.plot(date_list, max_temperature_values, label='Max Temp (°C)', marker='o')
    axis.plot(date_list, min_temperature_values, label='Min Temp (°C)', marker='o')
    axis.set_title('Sydney Temperature Trends')
    axis.set_xlabel('Date')
    axis.set_ylabel('Temperature (°C)')
    axis.legend()
    plt.grid(True)

    if display_mode == 'figure':
        return fig
    else:
        plt.show(block=False)
        plt.pause(0.001)

In [11]:
def plot_precipitation_trends(sydney_weather, display_mode='display'):
    """
    Visualize Sydney's precipitation trends based on the provided weather data.

    - sydney_weather: dictionary containing 'weather' list from fetch_weather_for_city() function.
    - display_mode: either 'display' to show the plot immediately or 'figure' to return the figure object.
    """
    date_list = []
    daily_precipitation_totals = []

    for daily_record in sydney_weather['weather']:
        date_list.append(daily_record['date'])
        # Calculate total precipitation by summing hourly precipitation values
        total_daily_precip = sum(float(hourly_data['precipMM']) for hourly_data in daily_record['hourly'])
        daily_precipitation_totals.append(total_daily_precip)

    fig, axis = plt.subplots()
    axis.bar(date_list, daily_precipitation_totals, color='skyblue')
    axis.set_title('Sydney Precipitation Trends')
    axis.set_xlabel('Date')
    axis.set_ylabel('Precipitation (mm)')
    plt.grid(True)

    if display_mode == 'figure':
        return fig
    else:
        plt.show(block=False)
        plt.pause(0.001)

In [ ]:
def plot_temp_humidity_combo(weather_data):
    days = [day['date'] for day in weather_data['weather']]
    max_temps = [int(day['maxtempC']) for day in weather_data['weather']]
    avg_humidity = [sum(float(hr['humidity']) for hr in day['hourly']) / len(day['hourly']) for day in weather_data['weather']]

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax1.bar(days, avg_humidity, color='skyblue', alpha=0.6, label='Avg Humidity (%)')
    ax1.set_ylabel('Average Humidity (%)', color='blue')

    ax2 = ax1.twinx()
    ax2.plot(days, max_temps, color='red', marker='o', label='Max Temp (°C)')
    ax2.set_ylabel('Temperature (°C)', color='red')

    plt.title("Daily Average Humidity vs. Max Temperature")
    fig.tight_layout()
    plt.show(block=False)
    plt.pause(0.001)

## 🤖 Natural Language Processing

In [12]:
def extract_weather_query_components(user_question):
    """
    Extract key components from a natural language weather question:
    location, attribute, and time period.

    - Defaults location to 'Sydney' if not specified.
    - Recognizes attributes like temperature, rain, precipitation, humidity, wind.
    - Recognizes time periods: today, tomorrow, day after tomorrow.
    """
    question_text = user_question.lower()
    possible_attributes = ['temperature', 'rain', 'precipitation', 'humidity', 'wind']
    possible_time_periods = ['today', 'tomorrow', 'day after tomorrow']

    found_attribute = None
    found_time_period = 'today'
    found_location = 'Sydney'

    for attribute in possible_attributes:
        if attribute in question_text:
            found_attribute = attribute
            break

    for time_period in possible_time_periods:
        if time_period in question_text:
            found_time_period = time_period
            break

    words = question_text.split()
    if 'in' in words:
        index_of_in = words.index('in')
        if index_of_in + 1 < len(words):
            found_location = ' '.join(words[index_of_in + 1:]).strip()

    return {
        'location': found_location,
        'attribute': found_attribute,
        'time_period': found_time_period
    }

## 🧩 Main Application Logic

In [14]:
# Tie everything together here
def formulate_weather_reply(parsed_input, sydney_weather_data):
    """
    Generate a natural language weather response based on parsed input and Sydney weather data.

    Args:
        parsed_input (dict): Contains keys 'location', 'attribute', 'time_period'
        sydney_weather_data (dict): Weather forecast data for Sydney

    Returns:
        str: Formulated response string
    """
    city = parsed_input['location']
    weather_feature = parsed_input['attribute']
    forecast_day = parsed_input['time_period']

    day_index = 0
    if forecast_day == 'tomorrow':
        day_index = 1
    elif forecast_day == 'day after tomorrow':
        day_index = 2

    try:
        forecast_for_day = sydney_weather_data['weather'][day_index]

        if weather_feature in ['temperature', None]:
            max_temperature = forecast_for_day['maxtempC']
            min_temperature = forecast_for_day['mintempC']
            return f"In {city.title()} on {forecast_for_day['date']}, the temperature will range from {min_temperature}°C to {max_temperature}°C."

        elif weather_feature in ['rain', 'precipitation']:
            total_precipitation = sum(float(hour['precipMM']) for hour in forecast_for_day['hourly'])
            return f"In {city.title()} on {forecast_for_day['date']}, the total precipitation will be {total_precipitation:.1f} mm."

        elif weather_feature == 'humidity':
            humidity_value = forecast_for_day['hourly'][0]['humidity']
            return f"In {city.title()} on {forecast_for_day['date']}, the humidity will be {humidity_value}%."

        elif weather_feature == 'wind':
            wind_speed = forecast_for_day['hourly'][0]['windspeedKmph']
            return f"In {city.title()} on {forecast_for_day['date']}, the wind speed will be {wind_speed} km/h."

        else:
            return "Sorry, I couldn't understand the weather attribute you asked for."

    except Exception as error:
        return f"Sorry, I couldn't retrieve the weather information: {error}"

## 🧭 User Interface

In [13]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧪 Testing and Examples

In [15]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.